<a href="https://colab.research.google.com/github/yagiyuki/clip-study-playground/blob/main/CLIP_acceleration_compile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIPで大量に画像分類するときの高速化実装の工夫

In [4]:
%%bash
# 画像を管理するパスを作成
mkdir -p data

# dataディレクトリにsample.jpegと言う名前で画像ファイルを上げる

In [5]:
%%bash
# 画像を999枚複製（合計1000枚）
for i in {1..999}; do
  cp data/sample.jpeg data/sample_${i}.jpeg
done

## 特に工夫なしの場合

In [3]:
%%time

import io
import requests
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import glob

# デバイス設定（そのままFP32, コンパイルなし）
device = 'cuda' if torch.cuda.is_available() else 'cpu'
HF_MODEL_PATH = 'line-corporation/clip-japanese-base'

# トークナイザ／プロセッサ／モデルの読み込み（標準設定）
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
processor = AutoImageProcessor.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
model = AutoModel.from_pretrained(HF_MODEL_PATH, trust_remote_code=True).to(device)

# 画像ファイルのリストを取得
image_list = glob.glob('data/*')

CPU times: user 1.25 s, sys: 69.2 ms, total: 1.32 s
Wall time: 3.7 s


In [4]:
%%time

# 各画像ごとに（バッチもAMPも事前計算もなしで）処理
for image_path in image_list:
    # テキストをその都度トークナイズ
    text_inputs = tokenizer(
        ["ベンチプレス", "スクワット", "デッドリフト"]
    ).to(device)

    # 画像を読み込んで前処理
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=[image], return_tensors="pt").to(device)

    with torch.no_grad():
        # テキスト特徴量を毎回計算
        text_features = model.get_text_features(**text_inputs)  # FP32

        # 画像特徴量を毎回計算
        image_features = model.get_image_features(inputs.pixel_values)  # FP32

        # 類似度計算・確率化
        text_probs = (image_features @ text_features.T).softmax(dim=-1)

    # 結果表示
    #print(f'Label probs of {image_path}:', text_probs)


CPU times: user 42.8 s, sys: 389 ms, total: 43.2 s
Wall time: 46.9 s


## torch.compileによるモデル最適化

In [5]:
%%time
import glob
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer

# デバイス設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
HF_MODEL_PATH = 'line-corporation/clip-japanese-base'

# モデル読み込み＋コンパイル
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
processor = AutoImageProcessor.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
model = AutoModel.from_pretrained(HF_MODEL_PATH, trust_remote_code=True).to(device)
model = torch.compile(model)  # 追加

CPU times: user 3.4 s, sys: 125 ms, total: 3.52 s
Wall time: 5.87 s


In [6]:
%%time
# 単純な推論ループ
with torch.no_grad():
    text = tokenizer(["ベンチプレス", "スクワット", "デッドリフト"]).to(device)
    for path in glob.glob('data/*'):
        img = Image.open(path).convert("RGB")
        inputs = processor(images=[img], return_tensors="pt").to(device)

        # 毎回テキスト特徴量を計算
        text_feats = model.get_text_features(**text)
        img_feats  = model.get_image_features(inputs.pixel_values)
        probs = (img_feats @ text_feats.T).softmax(dim=-1)
        #print(path, probs)

CPU times: user 40.1 s, sys: 174 ms, total: 40.3 s
Wall time: 42 s
